In [1]:
import sys
sys.path.insert(0, '/home/oscarwu/code/chemprop_developing')

import pickle as pkl
import numpy as np
import pandas as pd

from chemprop.features.utils import load_features, load_valid_atom_or_bond_features

from chemprop.args import TrainArgs
from chemprop.models.model import MoleculeModel

import torch
from chemprop.utils import load_checkpoint, load_scalers
from chemprop.features import set_extra_bond_fdim

from rdkit import Chem
from chemprop.rdkit import make_mol

import shap
from shap import PermutationExplainer
from copy import deepcopy


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# Create a MoleculeModel Wrapper
class MoleculeModelWrapper:
    def __init__(self, model, train_args, scalers, smiles, shap, features_batch, 
                 atom_descriptors_batch, atom_features_batch, bond_descriptors_batch, bond_features_batch):
        self.model = model.eval()
        self.train_args = train_args
        self.target_scaler, self.mol_feature_scaler, _, _, _ = scalers
        self.smiles = smiles
        self.shap = shap
        if features_batch is not None:
            self.features_batch = [self.mol_feature_scaler.transform(feat) for feat in features_batch]
        else:
            self.features_batch = None      
        self.atom_descriptors_batch = atom_descriptors_batch
        self.atom_features_batch = atom_features_batch
        self.bond_descriptors_batch = bond_descriptors_batch
        self.bond_features_batch = bond_features_batch
        
        self.batch = None
        self.extra_keep_features_batch = None
        self.extra_atom_keep_descriptors_batch = None
        self.extra_bond_keep_descriptors_batch = None
        self.extra_atom_keep_features_batch = None
        self.extra_bond_keep_features_batch = None
        self.chemprop_atom_keep_features = None
        self.chemprop_bond_keep_features = None
        

    def __call__(self, feature_choices):
        if isinstance(feature_choices, np.ndarray):
            if len(feature_choices.shape) == 1:
                feature_choices = feature_choices.reshape(1, -1)
                
        result = []
        
        for feature_choice in feature_choices:
        
            self.batch = [[make_mol(s=smi, keep_h=True, add_h=True, keep_atom_map=True)] for smi in self.smiles]
            self.extra_keep_features_batch = [feature_choice[0:20]] # 20 extra molecular features
            self.extra_atom_keep_descriptors_batch = [feature_choice[20:33]] # 13 atom descriptors
            self.extra_bond_keep_descriptors_batch = None # 0 bond descriptors
            self.extra_atom_keep_features_batch = None # 0 atom features
            self.extra_bond_keep_features_batch = [feature_choice[33:37]] # 4 bond features
            self.chemprop_atom_keep_features = feature_choice[37:45] # 8 chemprop atom features
            self.chemprop_bond_keep_features = feature_choice[45:49] # 4 chemprop bond features
            
            # this is from forward in original MoleculeModel in model.py
            output = self.model(batch=self.batch, 
                                features_batch=self.features_batch, 
                                atom_descriptors_batch=self.atom_descriptors_batch, 
                                atom_features_batch=self.atom_features_batch, 
                                bond_descriptors_batch=self.bond_descriptors_batch, 
                                bond_features_batch=self.bond_features_batch, 
                                constraints_batch=None,
                                bond_types_batch=None,
                                shap=self.shap, 
                                extra_keep_features_batch=self.extra_keep_features_batch, 
                                extra_atom_keep_descriptors_batch=self.extra_atom_keep_descriptors_batch, 
                                extra_bond_keep_descriptors_batch=self.extra_bond_keep_descriptors_batch, 
                                extra_atom_keep_features_batch=self.extra_atom_keep_features_batch, 
                                extra_bond_keep_features_batch=self.extra_bond_keep_features_batch, 
                                chemprop_atom_keep_features=self.chemprop_atom_keep_features, 
                                chemprop_bond_keep_features=self.chemprop_bond_keep_features)
        
            ## uncomment this if you want to scale back the target
            # xform = self.target_scaler.inverse_transform(output.item()).item()
            # xform = np.array(xform, ndmin=2)
            # result.append(xform)

            result.append(output.item()) # not scaled prediction for target to ensure same basis for shap values
        
        return np.array(result).reshape(len(feature_choices), -1)
        

In [3]:
def binary_masker(binary_mask, x):
    masked_x = deepcopy(x)
    masked_x[binary_mask == 0] = 0
    return np.array([masked_x])

In [4]:
df_smi = pd.read_csv('esol.csv')
smiles = df_smi['smiles'].values.tolist()

In [5]:
bond_features = load_valid_atom_or_bond_features('esol_wb97xd_bond_RBF_features.pkl', smiles)
atom_descriptors = load_valid_atom_or_bond_features('esol_wb97xd_atom_RBF_features.pkl', smiles)
mol_features = load_features("esol_wb97xd_molecule_features.csv")

In [6]:
bond_features_dict = {smi: bf for smi, bf in zip(smiles, bond_features)}
atom_descriptors_dict = {smi: ad for smi, ad in zip(smiles, atom_descriptors)}
mol_features_dict = {smi: mf.reshape(1, -1) for smi, mf in zip(smiles, mol_features)}

In [8]:
def main(smi, bond_features_dict=bond_features_dict, atom_descriptors_dict=atom_descriptors_dict, mol_features_dict=mol_features_dict):
    # Load TrainArgs and MoleculeModel
    path = "/home/oscarwu/code/chemprop_developing/developing/shap_v1/data/model.pt"
    state = torch.load(path, map_location=lambda storage, loc: storage)
    train_args = TrainArgs()
    train_args.from_dict(vars(state["args"]), skip_unsettable=True)
    set_extra_bond_fdim(train_args.bond_features_size)
    model = load_checkpoint(path)
    scalers = load_scalers(path)

    bond_features = [bond_features_dict[smi]]
    atom_descriptors = [atom_descriptors_dict[smi]]
    mol_features = [mol_features_dict[smi]]

    model_wrapper = MoleculeModelWrapper(model=model, 
                                     train_args=train_args, 
                                     scalers=scalers, 
                                     smiles=[smi], 
                                     shap=True, 
                                     features_batch=mol_features, 
                                     atom_descriptors_batch=atom_descriptors, 
                                     atom_features_batch=None, 
                                     bond_descriptors_batch=None, 
                                     bond_features_batch=bond_features)
    
    explainer = PermutationExplainer(model=model_wrapper, masker=binary_masker)
    feature_choice = np.array([[1]*49])
    unscaled_prediction = model_wrapper(feature_choice)
    explanation = explainer(feature_choice, max_evals=1000)

    return unscaled_prediction, explanation.values, explanation.base_values


In [9]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

In [10]:
def parallel_process(smiles_list):
    with ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(main, smiles_list), total=len(smiles_list)))
    return results

In [11]:
results = parallel_process(smiles)

  0%|          | 0/1040 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".

Loading pretrained parameter "encoder.encoder.0.W_i.weight".Loading pretrained parameter "encoder.encoder.0.W_i.weight".Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".Loading pretrained parameter "encoder.encoder.0.W_i.weight".Loading pretrained parameter "encoder.encoder.0.W_h.weight".

Loading pretrained 

PermutationExplainer explainer: 2it [03:38, 218.45s/it]              
PermutationExplainer explainer: 2it [03:47, 227.70s/it]              


Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.weight".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".
Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.

PermutationExplainer explainer: 2it [04:01, 241.26s/it]              


...


 92%|█████████▏| 961/1040 [1:27:06<07:04,  5.37s/it]


Loading pretrained parameter "readout.7.bias".


PermutationExplainer explainer: 2it [03:26, 206.32s/it]              
PermutationExplainer explainer: 2it [04:40, 280.60s/it]              
PermutationExplainer explainer: 2it [04:44, 284.38s/it]              


Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.weight".
Loading pretrained parameter "encoder.encoder.0.atom_descriptors_layer.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Loading pretrained parameter "readout.7.weight".
Loading pretrained parameter "readout.7.bias".


PermutationExplainer explainer: 2it [05:24, 324.96s/it]              
PermutationExplainer explainer: 2it [05:40, 340.32s/it]              
PermutationExplainer explainer: 2it [06:37, 397.08s/it]              
PermutationExplainer explainer: 2it [04:58, 298.16s/it]              
PermutationExplainer explainer: 2it [03:03, 183.43s/it]              
PermutationExplainer explainer: 2it [07:20, 440.48s/it]              
PermutationExplainer explainer: 2it [04:05, 245.37s/it]              
PermutationExplainer explainer: 2it [03:54, 234.29s/it]              
PermutationExplainer explainer: 2it [06:00, 360.41s/it]              
PermutationExplainer explainer: 2it [07:05, 425.74s/it]              
PermutationExplainer explainer: 2it [04:55, 295.04s/it]              
PermutationExplainer explainer: 2it [04:14, 254.52s/it]              
PermutationExplainer explainer: 2it [04:20, 260.40s/it]              
PermutationExplainer explainer: 2it [03:56, 236.78s/it]              
PermutationExplainer

In [17]:
results_dict = {smi: result for smi, result in zip(smiles, results)}

with open('results_dict.pkl', 'wb') as f:
    pkl.dump(results_dict, f)

In [22]:
results[0]

(array([[0.20481464]]),
 array([[-2.43807388e-02, -1.61210649e-02, -6.68234071e-03,
         -1.32575244e-02, -9.23284143e-03, -1.36505067e-03,
         -3.01148556e-03, -1.00862667e-02, -9.38800722e-04,
          7.80238919e-03, -3.18232924e-06, -1.28869293e-03,
          6.57470338e-03,  1.23121440e-03,  1.52087240e-02,
          8.11534673e-03,  5.87034501e-02,  1.39436685e-02,
         -6.05901331e-03, -5.24152275e-02, -5.74188381e-03,
         -4.71091449e-02, -3.90200775e-02, -1.62657791e-01,
         -1.20986939e-01, -2.82822065e-02,  4.03541699e-03,
          4.43667248e-03, -4.82322834e-03,  1.09270539e-01,
          2.98358940e-02, -1.52510241e-02,  1.66323487e-02,
          8.24513622e-03,  8.34804174e-02, -1.32100747e-01,
          3.92828088e-02,  3.25579271e-02,  7.98170269e-03,
          5.83713397e-02, -2.31971871e-04,  1.55744750e-02,
          3.66265491e-02, -1.13666579e-02, -9.02488856e-03,
         -7.02380883e-03, -9.75245982e-04, -1.77386885e-02,
         -2.3238

In [20]:
results_dict[0]

KeyError: 0

In [26]:
all_values = []
all_base_values = []

for val, index in enumerate(results):
    test, values, base_values = index
    all_values.append(values)
    all_base_values.append(base_values)

# Convert lists to NumPy arrays for easier averaging
all_values = np.array(all_values)
all_base_values = np.array(all_base_values)

# Compute the averages
average_values = np.mean(all_values, axis=0)
average_base_values = np.mean(all_base_values, axis=0)

print("Average Explanation Values:\n", average_values)
print("Average Base Values:\n", average_base_values)

Average Explanation Values:
 [[-0.00482263 -0.00937565 -0.00961549 -0.00658987  0.00305359  0.00792672
  -0.00340259 -0.00797053  0.0013253   0.00644273  0.00014257 -0.00166819
   0.00915524  0.00803045  0.01252571  0.01448961  0.01223905  0.01350322
  -0.0216333  -0.0072697  -0.02799503 -0.02318391 -0.04374482 -0.11068841
  -0.06013031 -0.01327596 -0.0179021  -0.06190701 -0.05859499  0.06551229
   0.0131969  -0.01414199  0.01053044 -0.02227289 -0.01675863 -0.05448138
   0.00544552  0.01089421 -0.01380961  0.01999582  0.01835815  0.00967331
   0.01576503 -0.01339447 -0.00915751 -0.01211627 -0.01892256 -0.00380637
  -0.00521774]]
Average Base Values:
 [[0.41036616]]


In [28]:
average_values

array([[-0.00482263, -0.00937565, -0.00961549, -0.00658987,  0.00305359,
         0.00792672, -0.00340259, -0.00797053,  0.0013253 ,  0.00644273,
         0.00014257, -0.00166819,  0.00915524,  0.00803045,  0.01252571,
         0.01448961,  0.01223905,  0.01350322, -0.0216333 , -0.0072697 ,
        -0.02799503, -0.02318391, -0.04374482, -0.11068841, -0.06013031,
        -0.01327596, -0.0179021 , -0.06190701, -0.05859499,  0.06551229,
         0.0131969 , -0.01414199,  0.01053044, -0.02227289, -0.01675863,
        -0.05448138,  0.00544552,  0.01089421, -0.01380961,  0.01999582,
         0.01835815,  0.00967331,  0.01576503, -0.01339447, -0.00915751,
        -0.01211627, -0.01892256, -0.00380637, -0.00521774]])

In [29]:
average_base_values

array([[0.41036616]])